<a href="https://colab.research.google.com/github/jpscard/Deteccao-de-buracos-e-lombadas-com-yolov7/blob/main/v2_Buracos_e_Lombadas_com_YOLOv7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equipe X
## Detecção de Buracos e Lombadas com YOLOv7



# Instalação de dependencias

_(é preciso checar se GPU está ativa no ambiente)_

In [ ]:
# MONTAR DRIVE (opcional por enquanto)
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Download YOLOv7 repository and install requirements
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 42.76 MiB/s, done.
Resolving deltas: 100% (466/466), done.
/content/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.0 MB/s 


# Download do dataset formatado para YOLOv7 PyTorch


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="geWcSg5Zluxm24dTfeMJ")
project = rf.workspace("projetoyolo").project("personal_dataset")
dataset = project.version(1).download("yolov7")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 210 kB/s 
     |████████████████████████████████| 178 kB 6.7 MB/s 
     |████████████████████████████████| 138 kB 46.7 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 145 kB 57.9 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=63a0f1beb266051415328b1101dfdd3ddbd84aade868bf98f3ad6e80c21fea1b
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certif

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to personal_dataset-1 in yolov7pytorch:: 100%|██████████| 891/891 [00:00<00:00, 5413.04it/s]


# Iniciar com treinamento personalizado



In [ ]:
# download COCO starting checkpoint
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
# run this cell to begin training
%cd /content/yolov7
!python train.py --batch 20 --epochs 500 --data {dataset.location}/data.yaml --weights '/content/yolov7/best16_300.pt' --device 0 

/content/yolov7
YOLOR 🚀 v0.1-115-g072f76c torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=20, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='/content/yolov7/personal_dataset-1/data.yaml', device='0', entity=None, epochs=500, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.p5.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp3', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=20, upload_dataset=False, v5_metric=False, weights='/content/yolov7/best16_300.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_ep

# Avaliação

In [ ]:
# Run evaluation
!python detect.py --weights runs/train/exp/weights/best.pt --conf 0.19 --source {dataset.location}/arquivos_testee

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.19, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='/content/yolov7/personal_dataset-1/arquivos_testee', update=False, view_img=False, weights=['runs/train/exp/weights/best.pt'])
YOLOR 🚀 v0.1-115-g072f76c torch 1.12.1+cu113 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36487166 parameters, 6194944 gradients, 103.2 GFLOPS
 Convert model to Traced-model... 
 traced_script_module 

In [ ]:
# exibir todas as inferencias de testes

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/content/yolov7/runs/detect/exp/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1  

# Exportar pesos e resultados

In [ ]:
# optional, zip to download weights and results locally

!zip -r export2.zip runs/detect
!zip -r export2.zip runs/train/exp/weights/best.pt
!zip export2.zip runs/train/exp/*

  adding: runs/detect/ (stored 0%)
  adding: runs/detect/exp/ (stored 0%)
  adding: runs/detect/exp/IMG_20221026_095130.jpg (deflated 0%)
  adding: runs/detect/exp/IMG_6105.JPG (deflated 1%)
  adding: runs/detect/exp/Captura de Tela (10).png (deflated 4%)
  adding: runs/detect/exp/Captura de Tela (54).png (deflated 3%)
  adding: runs/detect/exp/Captura de Tela (42).png (deflated 5%)
  adding: runs/detect/exp/lombada3.png (deflated 3%)
  adding: runs/detect/exp/IMG_6098.JPG (deflated 1%)
  adding: runs/detect/exp/Captura de Tela (39).png (deflated 3%)
  adding: runs/detect/exp/IMG_20221027_123357953.jpg (deflated 6%)
  adding: runs/detect/exp/IMG_6137.JPG (deflated 4%)
  adding: runs/detect/exp/Captura de Tela (51).png (deflated 4%)
  adding: runs/detect/exp/IMG_20221026_095617.jpg (deflated 0%)
  adding: runs/detect/exp/IMG_20221026_100126.jpg (deflated 0%)
  adding: runs/detect/exp/lombada9.png (deflated 3%)
  adding: runs/detect/exp/Captura de Tela (13).png (deflated 4%)
  adding: ru